In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch
import numpy as np

In [ ]:
# ۱. آماده‌سازی داده‌ها (در قالب Dataset مخصوص Hugging Face)
data = {
    "text": ["من خیلی خوشحال هستم", "این عالی است", "خیلی خوب بود",
             "من خیلی غمگین هستم", "این بد است", "اصلا خوب نبود"],
    "label": [1, 1, 1, 0, 0, 0]
}
dataset = Dataset.from_dict(data)

In [ ]:
# ۲. توکنایزر (استفاده از توکنایزر آماده برای تبدیل متن به عدد)
# ما از توکنایزر distilbert استفاده می‌کنیم چون با معماری ما همخوان است
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=8)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [ ]:
# ۳. تعریف معماری کاستوم (از صفر - بدون وزن‌های قبلی)
# اینجا تعداد لایه‌ها و هدها را خودمان تعیین می‌کنیم
config = AutoConfig.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,          # مثبت و منفی
    n_layers=2,            # فقط ۲ لایه (بجای ۶ لایه پیش‌فرض)
    n_heads=4,             # فقط ۴ هد توجه
    dim=64,                # ابعاد مدل
    hidden_dim=128         # ابعاد Feed Forward
)

# ساخت مدل با وزن‌های تصادفی (Random Initialization)
model = AutoModelForSequenceClassification.from_config(config)

In [ ]:
# ۴. تنظیمات آموزش (Training Arguments)
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,      # ۱۰۰ اپوک
    per_device_train_batch_size=2,
    logging_steps=10,
    learning_rate=5e-5,
)

In [ ]:
# ۵. استفاده از Trainer (بجای نوشتن حلقه دستی model.fit)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# شروع آموزش
print("شروع آموزش در Hugging Face...")
trainer.train()

شروع آموزش در Hugging Face...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.694748
20,0.691692
30,0.691346
40,0.691746
50,0.693330
60,0.693432
70,0.691293
80,0.693644
90,0.693527
100,0.692795


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=300, training_loss=0.692579779624939, metrics={'train_runtime': 4.266, 'train_samples_per_second': 140.646, 'train_steps_per_second': 70.323, 'total_flos': 2055225600.0, 'train_loss': 0.692579779624939, 'epoch': 100.0})

In [ ]:
# ۶. تست مدل با داده جدید
test_text = ["من غمگین هستم"]
inputs = tokenizer(test_text, return_tensors="pt", padding=True, truncation=True)
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

print(f"جمله: {test_text[0]}")
print(f"احتمال (منفی، مثبت): {predictions.numpy()}")

جمله: من غمگین هستم
احتمال (منفی، مثبت): [[0.49960214 0.5003978 ]]
